In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential


from google.colab import drive
import numpy as np
import pandas as pd

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [2]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
file_path = "/content/gdrive/My Drive/shakespeare.txt"

with open(file_path,"r") as f:
  text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [7]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [8]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [9]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [10]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [11]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [12]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [13]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [16]:
print(X.shape, Y.shape)

(38700, 40, 75) (38700, 75)


Our RNN model will be quite simple.

In [17]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [21]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [22]:
char_rnn.fit(X,Y, epochs=20, batch_size=1024)

Epoch 1/20
38/38 [==============================] - 1s 18ms/step - loss: 3.2335
Epoch 2/20
38/38 [==============================] - 1s 15ms/step - loss: 2.7560
Epoch 3/20
38/38 [==============================] - 1s 15ms/step - loss: 2.2892
Epoch 4/20
38/38 [==============================] - 1s 15ms/step - loss: 2.0425
Epoch 5/20
38/38 [==============================] - 1s 15ms/step - loss: 1.9072
Epoch 6/20
38/38 [==============================] - 1s 15ms/step - loss: 1.7983
Epoch 7/20
38/38 [==============================] - 1s 15ms/step - loss: 1.7144
Epoch 8/20
38/38 [==============================] - 1s 15ms/step - loss: 1.6439
Epoch 9/20
38/38 [==============================] - 1s 15ms/step - loss: 1.5858
Epoch 10/20
38/38 [==============================] - 1s 15ms/step - loss: 1.5297
Epoch 11/20
38/38 [==============================] - 1s 15ms/step - loss: 1.4749
Epoch 12/20
38/38 [==============================] - 1s 15ms/step - loss: 1.4274
Epoch 13/20
38/38 [==================

Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [23]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
      probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [24]:
sim = sample_text(char_rnn,sample_length=500) 

In [20]:
print(sim)

ill in Dunsinane and will endure
    Our that scram sporr the cay loon live
    Con'serfarte, whichs have do ath head.
  DUNCAN. Giel as knock! To sil, or it, if ourtor's them?
  SEDSOA As ITHERER. Inhal, thomeiSt by ell with a hail adivers
    Those classed had promicefwiceris
    Withuf the qufselt the restle's bryor sleep,
    Of born awe hath spent-hat loke to is.
                          Exit.



SCENE IVIIICTARBERSIDYON  CITED MONLOS AUD FOR MANBE OR. ANE GEANY BY WORL MUNt ARE-CONI  DOPTRMOCI ISD Shy COPISTED ON COPIAI NOP
A C


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [30]:
import string

In [31]:
def contains_vowels(sequence):
  vowels = ["a", "e", "i", "o", "u"]
  return any([vowel in list(sequence) for vowel in vowels])

In [32]:
contains_vowels("gradient")

True

In [120]:
sequences = []
labels = []
for i in range(50000):
 char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
 seq = "".join(char_list)
 sequences.append(seq)
 labels.append(int(contains_vowels(seq)))

In [121]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [122]:
df.head()

,sequence,label
0,ubisn,1
1,xzltx,0
2,cuqaw,1
3,kenxg,1
4,wmzlr,0


Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [123]:
# Data Preprocessing
full_text = "".join(df['sequence'].tolist())
chars = sorted(list(set(full_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
X = np.zeros((df.shape[0], 5, len(chars)), dtype=np.uint8 )
for i, seq in enumerate(df['sequence'].tolist()):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
y = np.asarray(df['label']).reshape(-1,1)
print(X.shape, y.shape)

total chars: 26
(50000, 5, 26) (50000, 1)


In [124]:
# generate validation data
sequences = []
labels = []
for i in range(100):
 char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
 seq = "".join(char_list)
 sequences.append(seq)
 labels.append(int(contains_vowels(seq)))
df_valid = pd.DataFrame({"sequence": sequences, "label":labels})
X_valid = np.zeros((df_valid.shape[0], 5, len(chars)), dtype=np.uint8 )
for i, seq in enumerate(df_valid['sequence'].tolist()):
    for t, char in enumerate(seq):
        X_valid[i, t, char_indices[char]] = 1
y_valid = np.asarray(df_valid['label']).reshape(-1,1)
print(X_valid.shape, y_valid.shape)

(100, 5, 26) (100, 1)


In [172]:
# Model setup and training
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(64, return_sequences=True, input_shape=(5, len(chars))))
char_rnn.add(tfkl.Dropout(0.2))
char_rnn.add(tfkl.LSTM(32, return_sequences=True ))
char_rnn.add(tfkl.Dropout(0.2))
char_rnn.add(tfkl.Dense(1 ,activation="sigmoid"))

In [173]:
char_rnn.compile(loss='binary_crossentropy', 
                 optimizer=tfk.optimizers.RMSprop(lr=0.1),
                 metrics=['accuracy']) 

In [174]:
hist = char_rnn.fit(X,y, epochs=20, batch_size=256, validation_data=(X_valid, y_valid))  

Epoch 1/20
196/196 [==============================] - 2s 9ms/step - loss: 0.4447 - accuracy: 0.7407 - val_loss: 0.3235 - val_accuracy: 0.7780
Epoch 2/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3129 - accuracy: 0.8133 - val_loss: 0.3022 - val_accuracy: 0.8160
Epoch 3/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3123 - accuracy: 0.8137 - val_loss: 0.3110 - val_accuracy: 0.8160
Epoch 4/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3123 - accuracy: 0.8143 - val_loss: 0.3039 - val_accuracy: 0.8160
Epoch 5/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3118 - accuracy: 0.8137 - val_loss: 0.3085 - val_accuracy: 0.8160
Epoch 6/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3116 - accuracy: 0.8149 - val_loss: 0.3189 - val_accuracy: 0.7780
Epoch 7/20
196/196 [==============================] - 1s 6ms/step - loss: 0.3118 - accuracy: 0.8152 - val_loss: 0.3024 - val_accuracy: 0.8160
Epoch 